In [5]:
import pandas as pd
import numpy as np
import json, requests

In [6]:
### Pulled this data from https://www.climatetrace.org/inventory in 02/2022
#### Disclaimer from the source that should be noted: "All Sectors excluding Forest and Land Use Emissions"
#df = pd.read_csv("CSVs/climatetrace_emissions_by_subsector_timeseries_all.csv")
api_url = 'https://api.climatetrace.org/emissions_by_subsector_timeseries?'
request_response = requests.get(url=api_url)
json_response = json.loads(request_response.text)
df = pd.json_normalize(json_response)
df

,country_full,country,sector,subsector,start,end,emissions
0,Aruba,ABW,agriculture,cropland fires,2020-01-01,2021-01-01,NaN
1,Aruba,ABW,agriculture,cropland fires,2019-01-01,2020-01-01,NaN
2,Aruba,ABW,agriculture,cropland fires,2018-01-01,2019-01-01,NaN
3,Aruba,ABW,agriculture,cropland fires,2017-01-01,2018-01-01,NaN
4,Aruba,ABW,agriculture,cropland fires,2016-01-01,2017-01-01,NaN
...,...,...,...,...,...,...,...
107245,Zimbabwe,ZWE,waste,wastewater treatment and discharge,2014-01-01,2015-01-01,NaN
107246,Zimbabwe,ZWE,waste,wastewater treatment and discharge,2013-01-01,2014-01-01,NaN
107247,Zimbabwe,ZWE,waste,wastewater treatment and discharge,2012-01-01,2013-01-01,NaN
107248,Zimbabwe,ZWE,waste,wastewater treatment and discharge,2011-01-01,2012-01-01,NaN


In [7]:
###Pull data on Forestry & Land Use separately because it is not included above
#dfl = pd.read_csv("CSVs/climatetrace_forestryandlanduse.csv")
api_url = 'https://api.climatetrace.org/emissions_by_subsector_timeseries?sector=forests'
request_response = requests.get(url=api_url)
json_response = json.loads(request_response.text)
dfl = pd.json_normalize(json_response)
dfl

,country_full,country,sector,subsector,start,end,emissions
0,Aruba,ABW,forests,forest clearing,2020-01-01,2021-01-01,0.0
1,Aruba,ABW,forests,forest clearing,2019-01-01,2020-01-01,2.0
2,Aruba,ABW,forests,forest clearing,2018-01-01,2019-01-01,0.0
3,Aruba,ABW,forests,forest clearing,2017-01-01,2018-01-01,0.0
4,Aruba,ABW,forests,forest clearing,2016-01-01,2017-01-01,0.0
...,...,...,...,...,...,...,...
10995,Zimbabwe,ZWE,forests,scrubland fires,2014-01-01,2015-01-01,NaN
10996,Zimbabwe,ZWE,forests,scrubland fires,2013-01-01,2014-01-01,NaN
10997,Zimbabwe,ZWE,forests,scrubland fires,2012-01-01,2013-01-01,NaN
10998,Zimbabwe,ZWE,forests,scrubland fires,2011-01-01,2012-01-01,NaN


In [8]:
##Manipulations for climate trace data cleaning
dfb = pd.concat([df,dfl])
dfb = dfb.dropna()
dfb = dfb.drop(dfb[dfb['emissions']==0].index)
dfb = dfb.rename(columns = {'emissions':'Tonnes_C02'})
dfb = dfb.rename(columns = {'start':'Start_Date'})
dfb = dfb.rename(columns = {'end':'End_Date'})
dfb = dfb.rename(columns={'country_full':'Country_Name'})
dfb['year'] = pd.DatetimeIndex(dfb['Start_Date']).year
dfb = dfb.rename(columns = {'country':'Country_Code'})
dfb.year = dfb.year.astype(str)
dfb

,Country_Name,Country_Code,sector,subsector,Start_Date,End_Date,Tonnes_C02,year
55,Aruba,ABW,buildings,cooking,2020-01-01,2021-01-01,13300.0,2020
56,Aruba,ABW,buildings,cooking,2019-01-01,2020-01-01,13300.0,2019
57,Aruba,ABW,buildings,cooking,2018-01-01,2019-01-01,13300.0,2018
58,Aruba,ABW,buildings,cooking,2017-01-01,2018-01-01,13700.0,2017
59,Aruba,ABW,buildings,cooking,2016-01-01,2017-01-01,13000.0,2016
...,...,...,...,...,...,...,...,...
10990,Zimbabwe,ZWE,forests,scrubland fires,2019-01-01,2020-01-01,4886251.0,2019
10991,Zimbabwe,ZWE,forests,scrubland fires,2018-01-01,2019-01-01,7213639.0,2018
10992,Zimbabwe,ZWE,forests,scrubland fires,2017-01-01,2018-01-01,5895177.0,2017
10993,Zimbabwe,ZWE,forests,scrubland fires,2016-01-01,2017-01-01,4112753.0,2016


In [9]:
### Pulling in population data and country categories from https://data.worldbank.org/indicator/SP.POP.TOTL
wpop = pd.read_csv('CSVs/API_SP.POP.TOTL_DS2_en_csv_v2_3628828.csv',skiprows=4)
wpop

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,Unnamed: 65
0,Aruba,ABW,"Population, total",SP.POP.TOTL,54208.0,55434.0,56234.0,56699.0,57029.0,57357.0,...,102565.0,103165.0,103776.0,104339.0,104865.0,105361.0,105846.0,106310.0,106766.0,NaN
1,Africa Eastern and Southern,AFE,"Population, total",SP.POP.TOTL,130836765.0,134159786.0,137614644.0,141202036.0,144920186.0,148769974.0,...,547482863.0,562601578.0,578075373.0,593871847.0,609978946.0,626392880.0,643090131.0,660046272.0,677243299.0,NaN
2,Afghanistan,AFG,"Population, total",SP.POP.TOTL,8996967.0,9169406.0,9351442.0,9543200.0,9744772.0,9956318.0,...,31161378.0,32269592.0,33370804.0,34413603.0,35383028.0,36296111.0,37171922.0,38041757.0,38928341.0,NaN
3,Africa Western and Central,AFW,"Population, total",SP.POP.TOTL,96396419.0,98407221.0,100506960.0,102691339.0,104953470.0,107289875.0,...,370243017.0,380437896.0,390882979.0,401586651.0,412551299.0,423769930.0,435229381.0,446911598.0,458803476.0,NaN
4,Angola,AGO,"Population, total",SP.POP.TOTL,5454938.0,5531451.0,5608499.0,5679409.0,5734995.0,5770573.0,...,25107925.0,26015786.0,26941773.0,27884380.0,28842482.0,29816769.0,30809787.0,31825299.0,32866268.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,Kosovo,XKX,"Population, total",SP.POP.TOTL,947000.0,966000.0,994000.0,1022000.0,1050000.0,1078000.0,...,1807106.0,1818117.0,1812771.0,1788196.0,1777557.0,1791003.0,1797085.0,1788878.0,1775378.0,NaN
262,"Yemen, Rep.",YEM,"Population, total",SP.POP.TOTL,5315351.0,5393034.0,5473671.0,5556767.0,5641598.0,5727745.0,...,24473176.0,25147112.0,25823488.0,26497881.0,27168210.0,27834811.0,28498683.0,29161922.0,29825968.0,NaN
263,South Africa,ZAF,"Population, total",SP.POP.TOTL,17099836.0,17524533.0,17965733.0,18423157.0,18896303.0,19384838.0,...,52832659.0,53687125.0,54544184.0,55386369.0,56207649.0,57009751.0,57792520.0,58558267.0,59308690.0,NaN
264,Zambia,ZMB,"Population, total",SP.POP.TOTL,3070780.0,3164330.0,3260645.0,3360099.0,3463211.0,3570466.0,...,14465148.0,14926551.0,15399793.0,15879370.0,16363449.0,16853608.0,17351714.0,17861034.0,18383956.0,NaN


In [10]:
##Changes to the population dataframe so I can easily merge it with the other dataframe
pop = wpop.loc[:, ~wpop.columns.str.contains('^Unnamed')]
pop = pop.drop(columns=['Indicator Name','Indicator Code'])
pop = pop.rename(columns={'Country Code':'Country_Code'})
keepcolumns = ['Country Name','Country_Code']
value_columns = pop.columns.difference(keepcolumns)
pop = pd.melt(pop,id_vars=keepcolumns,value_vars=value_columns,var_name='Year',
        value_name='Population')
pop

,Country Name,Country_Code,Year,Population
0,Aruba,ABW,1960,54208.0
1,Africa Eastern and Southern,AFE,1960,130836765.0
2,Afghanistan,AFG,1960,8996967.0
3,Africa Western and Central,AFW,1960,96396419.0
4,Angola,AGO,1960,5454938.0
...,...,...,...,...
16221,Kosovo,XKX,2020,1775378.0
16222,"Yemen, Rep.",YEM,2020,29825968.0
16223,South Africa,ZAF,2020,59308690.0
16224,Zambia,ZMB,2020,18383956.0


In [11]:
###Merging the two dataframes and a quick per capita calculation
dfp = pd.merge(dfb,pop,how='inner',left_on=['Country_Code','year'],right_on=['Country_Code','Year'])
dfp.reset_index(drop=True,inplace=True)
dfp['Pounds_C02_PerCapita'] = dfp['Tonnes_C02']/dfp['Population']*2000
dfp = dfp.drop(columns=['Year','Country Name'])
dfp = dfp.drop(dfp[dfp['Population']==0].index)
dfp = dfp.drop(dfp[dfp['Population']==""].index)
dfp = dfp.drop(dfp[dfp['Country_Name']=='Eritrea'].index)
dfp

,Country_Name,Country_Code,sector,subsector,Start_Date,End_Date,Tonnes_C02,year,Population,Pounds_C02_PerCapita
0,Aruba,ABW,buildings,cooking,2020-01-01,2021-01-01,13300.0,2020,106766.0,249.142986
1,Aruba,ABW,buildings,residential commercial onsite heating,2020-01-01,2021-01-01,129383.0,2020,106766.0,2423.674203
2,Aruba,ABW,manufacturing,other manufacturing,2020-01-01,2021-01-01,61851.0,2020,106766.0,1158.627278
3,Aruba,ABW,power,electricity generation,2020-01-01,2021-01-01,699000.0,2020,106766.0,13094.056160
4,Aruba,ABW,transport,aviation,2020-01-01,2021-01-01,168033.0,2020,106766.0,3147.687466
...,...,...,...,...,...,...,...,...,...,...
27573,Zimbabwe,ZWE,forests,scrubland fires,2015-01-01,2016-01-01,7804267.0,2015,13814642.0,1129.854397
27574,San Marino,SMR,forests,forest clearing,2019-01-01,2020-01-01,107.0,2019,33864.0,6.319395
27575,San Marino,SMR,forests,forest clearing,2018-01-01,2019-01-01,37.0,2018,33784.0,2.190386
27576,San Marino,SMR,forests,forest clearing,2017-01-01,2018-01-01,20.0,2017,33671.0,1.187966


In [12]:
###Bringing in Groupings from the world bank to category and remove outliers
wg = pd.read_csv('CSVs/Metadata_Country_API_SP.POP.TOTL_DS2_en_csv_v2_3628828.csv')
wg = wg[['Country Code','Region','IncomeGroup']]
dfm = pd.merge(dfp,wg,how='left',left_on='Country_Code',right_on='Country Code')
dfm = dfm.drop(columns=['Country Code'])
dfm['Pop Size'] = np.select(
[dfm['Population']>500000],['Over 500000'],default='Small'
)
dfm

,Country_Name,Country_Code,sector,subsector,Start_Date,End_Date,Tonnes_C02,year,Population,Pounds_C02_PerCapita,Region,IncomeGroup,Pop Size
0,Aruba,ABW,buildings,cooking,2020-01-01,2021-01-01,13300.0,2020,106766.0,249.142986,Latin America & Caribbean,High income,Small
1,Aruba,ABW,buildings,residential commercial onsite heating,2020-01-01,2021-01-01,129383.0,2020,106766.0,2423.674203,Latin America & Caribbean,High income,Small
2,Aruba,ABW,manufacturing,other manufacturing,2020-01-01,2021-01-01,61851.0,2020,106766.0,1158.627278,Latin America & Caribbean,High income,Small
3,Aruba,ABW,power,electricity generation,2020-01-01,2021-01-01,699000.0,2020,106766.0,13094.056160,Latin America & Caribbean,High income,Small
4,Aruba,ABW,transport,aviation,2020-01-01,2021-01-01,168033.0,2020,106766.0,3147.687466,Latin America & Caribbean,High income,Small
...,...,...,...,...,...,...,...,...,...,...,...,...,...
27472,Zimbabwe,ZWE,forests,scrubland fires,2015-01-01,2016-01-01,7804267.0,2015,13814642.0,1129.854397,Sub-Saharan Africa,Lower middle income,Over 500000
27473,San Marino,SMR,forests,forest clearing,2019-01-01,2020-01-01,107.0,2019,33864.0,6.319395,Europe & Central Asia,High income,Small
27474,San Marino,SMR,forests,forest clearing,2018-01-01,2019-01-01,37.0,2018,33784.0,2.190386,Europe & Central Asia,High income,Small
27475,San Marino,SMR,forests,forest clearing,2017-01-01,2018-01-01,20.0,2017,33671.0,1.187966,Europe & Central Asia,High income,Small


In [13]:
##Push the merged data as a csv file so it can easily be referenced
dfm.to_csv('CSVs/C02_Emissions.csv', index=False)
